In [ ]:
# 2.27.21
# This 123NF driver has all the funcs, whereas driver_code_forPaper-blank.ipynb has only the 123NF for heatmap paper

In [ ]:

import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [ ]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
modelpath = filepath + "004_GB_IEEE123_OPAL.xls"

# load data is not actually used in assessment, but the feeder object builds variables to hold load info
loadfolder = filepath
loadpath = filepath + '004_123NF_PVpen100_nocloud_minutewise_whead.csv'
headerpath = filepath + '004_GB_IEEE123_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder object

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
#print('depths length=',len(depths))

# print list of first 10 buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network

In [ ]:
# for converting a .dot to .png
(graph,) = pydot.graph_from_dot_file('generated_figs/temp/actConfig_auto-CPP_seed3')
graph.write_png('generated_figs/temp/actConfig_auto-CPP_seed3.png')
(graph,) = pydot.graph_from_dot_file('generated_figs/temp/actConfig_auto-CPP_seed6')
graph.write_png('generated_figs/temp/actConfig_auto-CPP_seed6.png')

In [ ]:
# write busnames into a csv
# with open("123NF_busList.csv", 'w', newline='') as csvfile:
#     spamwriter = csv.writer(csvfile, delimiter='-',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerows(graph.nodes)

In [ ]:
# Print 3-phase node_index table
my_buses=list(node_index_map.keys())
my_list=list(node_index_map.values())
list1 = [(i+1) * 3-2 for i in my_list]
list2 = [(i+1) * 3-1 for i in my_list]
list3 = [(i+1) * 3-0 for i in my_list]
for my_buses,list1, list2,list3 in zip(my_buses,list1,list2,list3):
    print(my_buses,list1, list2,list3)
#Table format: [bus_name, phaseA idx, phaseB idx, phaseC idx]
# indices are shifted by 1 to be for MATLAB (starts at 1 not 0)

In [ ]:
# R is of size 387, which is 129*3, so set n=129
n=129
parmObj=hm.configParms()
foo=['PBC','PBC','PBC','PBC']
parmObj.set_ctrlTypes(foo)
parmObj.set_version(1) # 1 for PBC

A, B = hm.setupStateSpace(parmObj,fin_feeder, n,node_index_map,depths)
print('R = '+str(R))
print('X = '+str(X))
print('A = '+str(A))
print('B = '+str(B))
print(A.shape)
print(B.shape)

# write A and B matrices to csv
#np.savetxt("123NF_Amat.csv", A, delimiter=",")
#np.savetxt("123NF_Bmat.csv", B, delimiter=",")

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_123NF.csv', R, delimiter=',')
#np.savetxt('Xmat_123NF.csv', X, delimiter=',')

In [ ]:
# [Optional] run impedance-related functions
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
        
# -------------------- now we call functions: ---------------------------------------------
#print(depths)
# modify node names when change feeders
#plot_histogram_RX_ratios(fin_feeder, leaves_only = True)
print('Z between buses:')
print(np.around(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_37', 'bus_15',depths),2))
print('\nBus 49 Z to substation:')
print(np.around(imp.get_total_impedance_from_substation(fin_feeder, 'bus_49',depths),2))
print('\nR/X ratio of bus 49 to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder,'bus_49',depths))
print('R/X Ratios:')
print('\n67 to 79: ')
print(imp.get_RX_ratio_between_two_buses(fin_feeder, 'bus_67', 'bus_79',depths))

In [ ]:
# Rx ratio and phase coupling ratio informs bad branches
branches = vis.assign_network_branches(fin_feeder, substation_name)
for branch in branches:
    if 'bus_150'in branch:
        branch.remove('bus_150')
        branch.remove('bus_149')
rx_dic = {}
phase_ratios = vis.phaseCouplingPerNode(fin_feeder, depths, file_name)
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider substation nodes, node 650 and 651 for 13NF
for node_name in graphNodes_nosub:
    foo=list((imp.get_RX_ratio_tosubst(fin_feeder, node_name, depths)).values())
    rx_3ph=np.array(foo).astype('float')
    rx_3ph[rx_3ph==0]=np.nan # assign NaN to all zeros
    rx_dic[node_name]=np.nanmean(rx_3ph) # nanmean isgnores nans, now scalar metric for each node
    
    ph_3ph=phase_ratios[node_name].astype('float')
    ph_3ph[ph_3ph==0]=np.nan # assign NaN to all zeros
    # take mean across phases, then take mag of the complex scalar
    phase_ratios[node_name]=np.absolute(np.nanmean(ph_3ph)) # nanmean isgnores nans
    
vis.createColorMap(fin_feeder, phase_ratios, 'phase_coupling') # input values are scalar reals, one for each node
vis.createColorMap(fin_feeder, rx_dic, 'rx')

#------------ make mixed colormap -----------
normd_phase_ratios = {}
normd_rx = {}
avg_phase = np.nanmean(list(phase_ratios.values()))
avg_rx = np.nanmean(list(rx_dic.values()))

for key, val in phase_ratios.items():
    if not np.isnan(val):
        normd_phase_ratios[key] = val/avg_phase
for key, val in rx_dic.items():
    if not np.isnan(val):
        normd_rx[key] = val/avg_rx

mixed_vals = {}
for node in graphNodes_nosub:
    mixed_vals[node] = normd_phase_ratios[node] + normd_rx[node]
vis.createColorMap(fin_feeder, mixed_vals, 'mixed')
# colormaps show up in generated_figs folder

In [ ]:
# collect cr,cx,and d for each branch --> put into coupling paper table
# this code doenst account for single and double phase, averages across the zeros in those cases

node1='bus_68'
node2='bus_71'

ph_3ph=imp.get_phase_ratio_between_two_buses(fin_feeder, node1,node2, depths)
print('complex 3ph=',ph_3ph)
cr=np.real(ph_3ph)
cx=np.imag(ph_3ph)
phrat_rscalar=np.absolute(np.nanmean(ph_3ph)) # real scalar metric
print('real scalar=',phrat_rscalar)

rx_3ph=imp.get_RX_ratio_between_two_buses(fin_feeder, node1,node2, depths)
print(rx_3ph.values())

z_brhead_subst=imp.get_total_impedance_from_substation(fin_feeder, node1, depths)
import scipy.linalg as LA
u, s, vh=LA.svd(z_brhead_subst)
len2sub=s[0] # largest singular value is length metric for matrix

print('------------------')
print('d=',np.around(sum(rx_3ph.values())/len(rx_3ph),3))
print('cr=',np.around(np.nanmean(cr),3))
print('cx=',np.around(np.nanmean(cx),3))
print('length=',np.around(len2sub,3))

In [ ]:
# ---------- make table of branches, ratios, and num used ------------
#3/17: this code chunk not used yet, made table in coupling figs spreadsheet instead

# fix: make list of branches only branches with 3 phase lines, since single and double phase laterals wont be used much 

# print('branches table:') # format into a table with index and branch nodes
# i=0
# for br in branches:
#     print(i,',',br)
#     i+=1
    
num_used=np.full((1, len(graphNodes_nosub)),-1) # placeholder

print('bus | rx_rat | ph_rat | num_used')
print('large and small rx_rat:')
foo1=sorted(normd_rx, key=normd_rx.get, reverse=True)[:3] # get keys of 3 largest values of dict
for node in foo1:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

foo2=sorted(normd_rx, key=normd_rx.get, reverse=False)[:3] # get keys of 3 smallest values of dict
for node in foo2:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

print('large and small ph_rat:')
foo3=sorted(normd_phase_ratios, key=normd_phase_ratios.get, reverse=True)[:3] # get keys of 5 largest values of dict
for node in foo3:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

foo4=sorted(normd_phase_ratios, key=normd_phase_ratios.get, reverse=False)[:3] # get keys of 5 smallest values of dict
for node in foo4:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

print('large and small sum_rat:')
foo3=sorted(mixed_vals, key=mixed_vals.get, reverse=True)[:3] # get keys of 5 largest values of dict
for node in foo3:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

foo4=sorted(mixed_vals, key=mixed_vals.get, reverse=False)[:3] # get keys of 5 smallest values of dict
for node in foo4:
    print(node,np.around(normd_rx[node],3),np.around(normd_phase_ratios[node],3),-1)

    
# for node in graphNodes_nosub:
#     print(node,np.around(normd_phase_ratios[node],3),np.around(normd_rx[node],3),-1)

In [ ]:
Source.from_file('colorMap_rx') # display colormap to see good/bad branches 
Source.from_file('colorMap_phase_coupl') # display colormap to see good/bad branches
Source.from_file('colorMap_mixed') # display colormap to see good/bad branches  